In [6]:
!pip install http://download.pytorch.org/whl/cu90/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
!pip install torchvision
!pip install tqdm

ERROR: torch-1.0.0-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [8]:
!pip install onnx

In [1]:
import torch
torch.tensor([1,2,3]).to("cuda:0")

tensor([1, 2, 3], device='cuda:0')

In [2]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset,
                            DataLoader,
                            TensorDataset)
import tqdm

## Data 업로드

In [ ]:
from torchvision import models

def create_network():
    
    # resnet18 기반의 이종 분류 식별 모델
    net = models.resnet18()
    fc_input_dim = net.fc.in_features
    net.fc = nn.Linear(fc_input_dim, 2)
    return net

# 모델 생성
net = create_network()

# 파라미터 읽기 및 모델에 설정
prm = torch.load("taco_burrito.prm", map_location="cpu")
net.load_state_dict(prm)

# 평가 모드로 설정
net.eval()

In [ ]:
import torch.onnx
dummy_data = torch.empty(1, 3, 224, 224, dtype=torch.float32)
torch.onnx.export(net, dummy_data, "taco_burrito.onnx")

In [ ]:
import onnx
from caffe2.python.onnx import backend as caffe2_backend

# ONNX 모델 불러오기
onnx_model = onnx.load("/content/taco_burrito.onnx")

# ONNX 모델을 Caffe2 모델로 변환
backend = caffe2_backend.prepare(onnx_model)

In [ ]:
from PIL import Image
from torchvision import transforms

# 이미지를 잘라서 Tensor로 변환하는 함수
transform = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# 이미지 불러오기
img = Image.open("/content/360.jpg")

# Tensor로 변환해서 배치 차원을 더함
img_tensor = transform(img).unsqueeze(0)

# ndarray로 변환
img_ndarray = img_tensor.numpy()

# 파이토치로 실행
net(img_tensor)

In [ ]:
# ONNX/Caffe2로 실행
output = backend.run(img_ndarray)
output[0]

In [ ]:
from caffe2.python.onnx.backend import Caffe2Backend

init_net, predict_net = \
    Caffe2Backend.onnx_graph_to_caffe2_net(onnx_model)

In [ ]:
with open('init_net.pb', "wb") as fopen:
    fopen.write(init_net.SerializeToString())

with open('predict_net.pb', "wb") as fopen:
    fopen.write(predict_net.SerializeToString())